In [2]:
import pandas as pd
import json

In [11]:
# import the data in a dataframe
json_data=open('nuclear_tests.json').read()
data = json.loads(json_data)
df = pd.DataFrame(data)
df.head()

,body wave magnitude,country,date,decimal latitude/longitude,name,original time,site,type,yield
0,4.9,CHN,29/07/1996,41.82/88.42,-,01:48:57 (Universal Coordinated Time),Lop Nor,Underground,NaN
1,5.9,CHN,08/06/1996,41.66/88.69,-,02:55:58 (Universal Coordinated Time),Lop Nor,Underground,NaN
2,6,CHN,17/08/1995,41.56/88.8,-,00:59:57 (Universal Coordinated Time),Lop Nor,Underground,NaN
3,6.1,CHN,15/05/1995,41.6/88.82,-,04:05:57 (Universal Coordinated Time),Lop Nor,Underground,NaN
4,6,CHN,07/10/1994,41.66/88.75,-,03:25:58 (Universal Coordinated Time),Lop Nor,Underground,NaN
